In [ ]:
import re
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
from spellchecker import SpellChecker

In [ ]:
spell = SpellChecker()

In [ ]:
filename = "elan_dataset_merged.txt"

In [ ]:
output_filename = "elan_cleaned_retained_merged.txt"

In [ ]:
def get_token(gloss, remove=False):
    
    removed_decorator_start = ""
    removed_decorator_end = ""

    if gloss[:8] == "FBUOY:DS":
        split = gloss.split(':')
        if len(split) > 2:
            gloss = gloss.split(':')[2]
            removed_decorator_start = "FBUOY "
    elif gloss[:6] == "FBUOY:":
        split = gloss.split(':')
        if len(split) > 1:
            gloss = gloss.split(':')[1]
            removed_decorator_start = "FBUOY "
    elif gloss[:6] == "FUBOY:":
        split = gloss.split(':')
        if len(split) > 1:
            gloss = gloss.split(':')[1]
            removed_decorator_start = "FBUOY "
    elif gloss[:6] == "FBOUY:":
        split = gloss.split(':')
        if len(split) > 1:
            gloss = gloss.split(':')[1]
            removed_decorator_start = "FBUOY "
    elif gloss[:6] == "GICA):":
        split = gloss.split(':')
        if len(split) > 1:
            gloss = gloss.split(':')[1]
    elif gloss[:6] == "TBUOY:":
        split = gloss.split(':')
        if len(split) > 1:
            gloss = gloss.split(':')[1]
            removed_decorator_start = "FBUOY "
    elif gloss[:3] == "FS:":
        gloss = gloss[3:]
        removed_decorator_start = "FS "
    elif gloss[:3] == "FB:":
        gloss = gloss[3:]
        removed_decorator_start = "FB "
    elif gloss[:2] == "DS":
        split = gloss.split(':')
        if len(split) > 1:
            gloss = gloss.split(':')[1]
            removed_decorator_start = "DS "
    elif gloss[:2] == "G(":
        split = gloss.split(':')
        if len(split) > 1:
            gloss = gloss.split(':')[1]
    elif gloss[:3] == "CA:":
        split = gloss.split(':')
        if len(split) > 1:
            gloss = gloss.split(':')[1]
    elif gloss[:3] == "GA:":
        split = gloss.split(':')
        if len(split) > 1:
            gloss = gloss.split(':')[1]
    elif gloss[:2] == "G:":
        split = gloss.split(':')
        if len(split) > 1:
            gloss = gloss.split(':')[1]
    elif gloss[:2] == "M:":
        split = gloss.split(':')
        if len(split) > 1:
            gloss = gloss.split(':')[1]
    elif gloss[:7] == "FINISH.":
        gloss = "FINISH"

    index = gloss.find("-2H")
    if index != -1:
        gloss = gloss[:index]
        removed_decorator_end = " 2H"
    index = gloss.find("-1H")
    if index != -1:
        gloss = gloss[:index]
        removed_decorator_end = " 1H"
    index = gloss.find("2-H")
    if index != -1:
        gloss = gloss[:index]
        removed_decorator_end = " 2H"
    index = gloss.find("1-H")
    if index != -1:
        gloss = gloss[:index]
        removed_decorator_end = " 1H"

    index = gloss.find("(")
    if index != -1:
        gloss = gloss[:index]
    
    if remove:
        return gloss
    return f'{removed_decorator_start}{gloss}{removed_decorator_end}'

In [ ]:
def clean_sentence(sentence):
    
    sentence = sentence.replace("'s", " is")
    sentence = sentence.replace("n't", " not")
    sentence = sentence.replace("'d", " had")
    sentence = sentence.replace("'ll", " will")
    sentence = sentence.replace("'m", " am")
    sentence = sentence.replace("'ve", " have")
    sentence = sentence.replace("'re", " are")
    sentence = sentence.replace("ahh", "")
    sentence = sentence.replace("umm", "")
    sentence = sentence.replace("?", " ?")
    sentence = sentence.replace("(", "")
    sentence = sentence.replace(")", "")



    cleaned_sentence = ""
    
    for char in sentence:
        if char.isalpha() or char == "'" or char == "?":
            cleaned_sentence += char
        else:
            cleaned_sentence += " "
    
    cleaned_sentence = cleaned_sentence.lower()
    cleaned_sentence = re.sub(' +', ' ', cleaned_sentence)
    
    return cleaned_sentence.strip()

def should_skip_token(token):
    if token[:3] == "PT:":
        return True
    if token[:5] == "LOOK(":
        return True
    if token[:7] == "PTBUOY:":
        return True
        
    return False
    
def clean_tokens(token_sequence):
    
    token_sequence = token_sequence.replace("FALSE-START", "")
    token_sequence = token_sequence.replace("FALSE START", "")
    token_sequence = token_sequence.replace("?", " ?")
    tokens = token_sequence.strip().split(" ")
    
    
    cleaned_tokens = []
        
    for token in tokens:
        if token[:3] == "PT:":
            continue
        if token[:5] == "LOOK(":
            continue
        if token[:7] == "PTBUOY:":
            continue
        
        token = get_token(token)
        
        if token in ["WELL", "", "AHH", "UMM", "FSL", "PTBUOY", "HMM", "ERR", "PHOOEY", "INDETERMINATE", "INDECIPHERABLE"]:
            continue

        if token[-1].isdigit():
            token = token[:-1]

        if len(token) < 2:
            continue
                        
        cleaned_tokens.append(token)
        
    unduped_tokens = []
    
    for i in range(0, len(cleaned_tokens)):
        if (i == 0 or cleaned_tokens[i] != cleaned_tokens[i - 1]) and (i < 2 or cleaned_tokens[i] != cleaned_tokens[i - 2]):
            unduped_tokens.append(cleaned_tokens[i].replace("-", " "))
        
    return (" ".join(unduped_tokens)).upper()

def spell_check(sequence, upper):
    sequence = sequence.split(" ")
    output = []
    for word in sequence:
        if upper:
            if spell.correction(word):
                output.append(spell.correction(word).upper())
            else:
                output.append(word)
        else:
            if spell.correction(word):
                output.append(spell.correction(word))
            else:
                output.append(word)
    return " ".join(output)

def create_output_file(input_filename, output_filename):
    input_file = open(input_filename, "r")
    output_file = open(output_filename, "w+")
    lines = input_file.readlines()
    
    cleaned_lines = []
        
    for line in lines:
        split = line.split("\t")
        
        sentence = clean_sentence(clean_sentence(split[0]))
        if len(sentence) == 0:
            continue
#         sentence = spell_check(sentence, False)

        
        tokens = clean_tokens(clean_tokens(split[1]))
        if len(tokens) == 0:
            continue
#         tokens = spell_check(tokens, True)
        
        if (len(sentence) == 0 or len(tokens) == 0) or sentence.count(" ") > 25 or tokens.count(" ") > 25:
            continue
            
        print(split[1])
        print(tokens)
        print()
        
        cleaned_lines.append(f'{clean_sentence(split[0])}\t{clean_tokens(split[1])}\n')
        
    cleaned_lines.sort(key= lambda line: (line.split("\t")[0].count(" ")))
    
    text = ""
    
    for cleaned_line in cleaned_lines:
        text += cleaned_line
        
    output_file.write(text)
    output_file.close()
    input_file.close()
    
    return cleaned_lines

In [ ]:
sentences = create_output_file(filename, output_filename)

In [ ]:
sentences

In [ ]:
def plot_hist(sentences, index, bins, title, ylabel, xlabel):
    sentence_lengths = []

    for line in sentences:
        sentence = line.split("\t")
        count = sentence[index].count(" ")
        sentence_lengths.append(count + 1)
    
    plt.figure(figsize=(14,7)) # Make it 14x7 inch
    plt.style.use('seaborn-whitegrid') # nice and clean grid

    n, bins, patches = plt.hist(sentence_lengths, bins=bins, facecolor='#2ab0ff', edgecolor='#e0e0e0', linewidth=0.5, alpha=0.7)

    n = n.astype('int') # it MUST be integer

    # Good old loop. Choose colormap of your taste
    for i in range(len(patches)):
        patches[i].set_facecolor(plt.cm.viridis(n[i]/max(n)))


    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()

In [ ]:
plot_hist(sentences, 0, 25, "", "", "")